In [1]:
import pandas as pd
import numpy as np

In [2]:
constituents = pd.read_csv("constituents.csv")
fundamentals = pd.read_csv("fundamentals.csv")
prices_split_adjusted = pd.read_csv("prices-split-adjusted.csv")
securities = pd.read_csv("securities.csv")

In [24]:
# sample portfolio
portfolio = {'AAPL': 0.5, 'MSFT': 0.5}

In [4]:
constituents.columns

Index(['Symbol', 'Name', 'Sector'], dtype='object')

In [8]:
constituents = constituents[['Symbol', 'Sector']]

In [9]:
fundamentals.columns

Index(['Unnamed: 0', 'Ticker Symbol', 'Period Ending', 'Accounts Payable',
       'Accounts Receivable', 'Add'l income/expense items', 'After Tax ROE',
       'Capital Expenditures', 'Capital Surplus', 'Cash Ratio',
       'Cash and Cash Equivalents', 'Changes in Inventories', 'Common Stocks',
       'Cost of Revenue', 'Current Ratio', 'Deferred Asset Charges',
       'Deferred Liability Charges', 'Depreciation',
       'Earnings Before Interest and Tax', 'Earnings Before Tax',
       'Effect of Exchange Rate',
       'Equity Earnings/Loss Unconsolidated Subsidiary', 'Fixed Assets',
       'Goodwill', 'Gross Margin', 'Gross Profit', 'Income Tax',
       'Intangible Assets', 'Interest Expense', 'Inventory', 'Investments',
       'Liabilities', 'Long-Term Debt', 'Long-Term Investments',
       'Minority Interest', 'Misc. Stocks', 'Net Borrowings', 'Net Cash Flow',
       'Net Cash Flow-Operating', 'Net Cash Flows-Financing',
       'Net Cash Flows-Investing', 'Net Income', 'Net Income Ad

In [10]:
fundamentals = fundamentals[['Ticker Symbol']]

In [11]:
prices_split_adjusted.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

In [12]:
# merge
universe = set(securities['Ticker symbol']) \
         & set(prices_split_adjusted['symbol']) \
         & set(fundamentals['Ticker Symbol']) \
         & set(constituents['Symbol'])
#securities = securities[securities['Ticker symbol'].isin(universe)]
#prices_split_adjusted = prices_split_adjusted[prices_split_adjusted['symbol'].isin(universe)]
#fundamentals = fundamentals[fundamentals['Ticker Symbol'].isin(universe)]
#constituents = constituents[constituents['Symbol'].isin(universe)]

{'AAL',
 'AAP',
 'AAPL',
 'ABBV',
 'ABC',
 'ABT',
 'ADBE',
 'ADI',
 'ADM',
 'ADS',
 'ADSK',
 'AEE',
 'AEP',
 'AFL',
 'AIG',
 'AIV',
 'AIZ',
 'AJG',
 'AKAM',
 'ALB',
 'ALK',
 'ALL',
 'ALLE',
 'ALXN',
 'AMAT',
 'AME',
 'AMG',
 'AMGN',
 'AMP',
 'AMT',
 'AMZN',
 'AN',
 'ANTM',
 'AON',
 'APA',
 'APC',
 'APD',
 'APH',
 'ARNC',
 'ATVI',
 'AVB',
 'AVGO',
 'AVY',
 'AWK',
 'AXP',
 'AYI',
 'AZO',
 'BA',
 'BAC',
 'BAX',
 'BBBY',
 'BBT',
 'BBY',
 'BCR',
 'BDX',
 'BHI',
 'BIIB',
 'BK',
 'BLL',
 'BMY',
 'BSX',
 'BWA',
 'BXP',
 'C',
 'CAG',
 'CAH',
 'CAT',
 'CB',
 'CBG',
 'CCI',
 'CCL',
 'CELG',
 'CERN',
 'CF',
 'CFG',
 'CHD',
 'CHK',
 'CHRW',
 'CHTR',
 'CI',
 'CINF',
 'CL',
 'CLX',
 'CMA',
 'CME',
 'CMG',
 'CMI',
 'CMS',
 'CNC',
 'CNP',
 'COF',
 'COG',
 'COL',
 'COO',
 'COST',
 'COTY',
 'CPB',
 'CRM',
 'CSCO',
 'CSRA',
 'CSX',
 'CTAS',
 'CTL',
 'CTSH',
 'CTXS',
 'CVS',
 'CVX',
 'CXO',
 'D',
 'DAL',
 'DD',
 'DE',
 'DFS',
 'DG',
 'DGX',
 'DHI',
 'DHR',
 'DIS',
 'DISCA',
 'DISCK',
 'DLPH',
 'DLR',
 'DLT

In [30]:
def historical_volatility(df):
    p = np.array(df.close)
    lr = np.log(p[1:]) - np.log(p[:-1])
    return np.sum(np.square(lr))
    
def covariance(df1, df2):
    date1 = np.array(df1.date)
    date2 = np.array(df2.date)
    start = max(date1[0], date2[0])
    end = min(date1[-1], date2[-1])
    p1 = np.array(df1[(start <= df1.date) & (df1.date <= end)].close)
    p2 = np.array(df2[(start <= df2.date) & (df2.date <= end)].close)
    lr1 = np.log(p1[1:]) - np.log(p1[:-1])
    lr2 = np.log(p2[1:]) - np.log(p2[:-1])
    return np.sum(lr1*lr2)

In [34]:
vol = []
portfolio_prices = {s: prices_split_adjusted[prices_split_adjusted.symbol == s].sort(['date'], ascending=[True]) for s in portfolio.keys()}
for s in universe:
    df = prices_split_adjusted[prices_split_adjusted.symbol == s]
    df.sort(['date'], ascending=[True], inplace=True)
    vol.append({
        'symbol': s,
        'volatility': historical_volatility(df),
        'added_volatility': sum(w * covariance(df, portfolio_prices[s]) for s, w in portfolio.items()),
    })
vol = pd.DataFrame(vol, columns = ['symbol', 'volatility', 'added_volatility'])
vol

/usr/local/lib/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app
/usr/local/lib/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/usr/local/lib/anaconda/lib/python3.5/site-packages/pandas/core/frame.py:3225: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  na_position=na_position)


,symbol,volatility,added_volatility
0,PWR,0.763973,0.176044
1,GGP,0.599776,0.165825
2,VRSK,0.279592,0.097770
3,LYB,0.848951,0.217989
4,AMP,0.631578,0.235986
5,ETFC,1.009742,0.272292
6,MON,0.438543,0.146210
7,ADSK,0.826802,0.247966
8,HRB,0.661296,0.140528
9,UDR,0.380966,0.137356


In [35]:
securities.columns

Index(['Ticker symbol', 'Security', 'SEC filings', 'GICS Sector',
       'GICS Sub Industry', 'Address of Headquarters', 'Date first added',
       'CIK'],
      dtype='object')

In [36]:
securities = securities[['Ticker symbol', 'GICS Sector', 'GICS Sub Industry']]

In [57]:
def merge(keep_key, dfs_by_key):
    merged = dfs_by_key[0][1].copy()
    for k, df in dfs_by_key[1:]:
        merged = merged.merge(df, left_on=dfs_by_key[0][0], right_on=k, how='inner')
        if k != keep_key:
            del merged[k]
    if dfs_by_key[0][0] != keep_key:
        del merged[dfs_by_key[0][0]]
    return merged

In [62]:
df = merge('symbol', [('symbol', vol), ('Symbol', constituents), ('Ticker symbol', securities)])
df

,symbol,volatility,added_volatility,Sector,GICS Sector,GICS Sub Industry
0,PWR,0.763973,0.176044,Industrials,Industrials,Industrial Conglomerates
1,GGP,0.599776,0.165825,Real Estate,Real Estate,Retail REITs
2,VRSK,0.279592,0.097770,Industrials,Industrials,Research & Consulting Services
3,LYB,0.848951,0.217989,Materials,Materials,Diversified Chemicals
4,AMP,0.631578,0.235986,Financials,Financials,Asset Management & Custody Banks
5,ETFC,1.009742,0.272292,Financials,Financials,Investment Banking & Brokerage
6,MON,0.438543,0.146210,Materials,Materials,Fertilizers & Agricultural Chemicals
7,ADSK,0.826802,0.247966,Information Technology,Information Technology,Application Software
8,HRB,0.661296,0.140528,Financials,Financials,Consumer Finance
9,UDR,0.380966,0.137356,Real Estate,Real Estate,Residential REITs


In case I don't finish, here is my ranking scheme:

Features:
 - sectors, industries, etc. (consider the existing fraction of the portfolio that is of that category)
 - the percentile of the historical volatility (sum of squares of log returns)
 - the percentile of the added volatility (sum of covariances weighted by the given security in the portfolio, based on portfolio variance that scales with the weight of the security being considered)

Percentiles are used since they are automatically normalized and are resistant to outliers (alternatively, z-scores could be used for symmetrically distributed features). They are calculated simply by dividing the rank of a feature by the total number of values.

The corresponding weights are determined by:
 - inspired by the Herfindahl index, $\Sigma_i (p_i - p_i^*)^2$, where $p_i$ is the proportion of category $i$ in the portfolio and $p_i^*$ is its proportion in the broader market
 - simply $\frac{1}{N}$ as this becomes less important with large $N$
 - the percentile of the portfolio variance

This quantity is then ranked so that this quantity is minimized.

This can be improved by adding features that determine the quality of the investment, for example:
 - the industry-adjusted P/E ratio

However these do not vary with the portfolio and aren't as reliable. Something that I think would be better is letting the user determine the weights for this section; this way they are customized for him/her.